In [1]:
import cv2
import time
import math
import carla
import weakref
import numpy as np
import threading
import time
import random

In [3]:
# Global dictionary to store GNSS data
gnss_data = {'lat': None, 'lon': None}
stop_flag = threading.Event()
log_file = "gnss_coordinates.txt"  # File to save coordinates

In [4]:
import sys
# sys.path.append('/home/kaustubh/CARLA_0_9_14/PythonAPI/carla')

In [2]:
client = carla.Client("localhost", 2000)
world = client.get_world()


In [3]:
map_ = world.get_map()

In [4]:
spawn_points = map_.get_spawn_points()

In [8]:
# Selecting the vehicle and spawning it
vehicle_bp = world.get_blueprint_library().filter('vehicle.mini.cooper_s')
# vehicle = world.get_actors().filter('*vehicle*')[0]
# camera = world.get_actors().filter('*camera*')[0]
# gnss = world.get_actors().filter('*gnss*')[0]
# collision_bp = world.get_blueprint_library().find('sensor.other.collision')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [68]:
map = world.get_map()
waypoints = [map.get_waypoint(spawn_points[0].location)]
for _ in range(50):  # Generate a simple route
    next_wps = waypoints[-1].next(10.0)
    waypoints.append(next_wps[0] if next_wps else waypoints[-1])
current_waypoint_idx = 0

In [69]:
for i in waypoints:
    world.debug.draw_point(
            i.transform.location,
            size=0.2,
            color=carla.Color(0, 255, 0),
            life_time=6.0
        )
    
    # print(i.transform.location)

In [ ]:
print(collision_bp)
camera_init_trans = carla.Transform(carla.Location(z=camera_pos_z, x=camera_pos_x))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [ ]:
# print(vehicle)
# print(camera)
# print(gnss)
for actor in world.get_actors():
    print(actor)

In [ ]:
# Simple Control
vehicle.apply_control(carla.VehicleControl(throttle=0, steer=1, brake=1,hand_brake=False))

In [ ]:
# Camera setup
# import carla.libcarla


# camera_pos_z = 3
# camera_pos_x = 0
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
# # camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
# camera_bp.set_attribute('image_size_x', '640')
# camera_bp.set_attribute('image_size_y', '360')

# camera_init_trans = carla.Transform(carla.Location(z=camera_pos_z, x=camera_pos_x))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

In [ ]:
PREFERRED_SPEED = 30
SPEED_THRESHOL = 2

# font = cv2.FONT_HERSHEY_SIMPLEX

# org = (30, 30)
# org2 = (30, 50)
# org3 = (30, 70)
# org4 = (30, 90)
# org5 = (30, 110)

# fontscale = 0.5

# color = (255, 255, 255)
# thickness = 1

# def maintain_spped(s):
#     if s >= PREFERRED_SPEED:
#         return 0
#     elif s < PREFERRED_SPEED - SPEED_THRESHOL:
#         return 0.8
#     else:
#         return 0.4

In [ ]:
import carla.libcarla


camera_pos_z = 2.5
camera_pos_x = 0.65
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
# camera_bp = world.get_blueprint_library().find('sensor.camera.instance_segmentation')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

# camera_init_trans = carla.Transform(carla.Location(z=camera_pos_z, x=camera_pos_x))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

quit = False

while True: 
    world.tick()
    
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    
    image = camera_data['image']
    
    # steering_angle = 0
    # # v = vehicle.get_velocity()
    # speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
    # # image = cv2.putText(image, 'Speed: ' + str(int(speed)) + ' kmh', org2,
    # #                     font, fontscale, color, thickness, cv2.LINE_AA)
    
    # estimate_throttle = maintain_spped(speed)
    # estimate_throttle = 10
    # vehicle.apply_control(carla.VehicleControl(throttle=1,
    #                                            steer=steering_angle))
    
    cv2.imshow('RGB Camera', image)
    
cv2.destroyAllWindows()
camera.stop()
# for actor in world.get_actors().filter('*vehicle*'):
#     actor.destroy()
# for sensor in world.get_actors().filter('*sensor*'):
#     sensor.destroy()

In [9]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [10]:
x = world.get_actors().filter('*vehicle*')

for i in x:
    print(i)

In [ ]:
# for sen in world.get_actors().filter('*gnss*'):
for sen in world.get_blueprint_library().filter('*lidar*'):
    print(sen)

In [ ]:
gnss = world.get_actors().filter('*gnss*')[0]

In [ ]:

# def _on_gnss_event(event):
#     # Update the global dictionary directly
#     gnss_data['latitude'] = event.latitude
#     gnss_data['longitude'] = event.longitude

# # Function to log GNSS data to a text file every 4 seconds for 1 minute
# def log_gnss_data():
#     start_time = time.time()
#     with open(log_file, "w") as file:  # Open file in write mode
#         while not stop_flag.is_set():
#             print
#             current_time = time.time()
#             elapsed = current_time - start_time

#             # Stop after 1 minute
#             if elapsed > 60:
#                 print("Logging complete.")
#                 break

#             if gnss_data['lat'] is not None and gnss_data['lon'] is not None:
#                 file.write(f"{gnss_data['lat']:.7f},{gnss_data['lon']:.7f}\n")
#                 file.flush()  # Immediately write to disk
#                 print(f"Logged: Latitude: {gnss_data['lat']}, Longitude: {gnss_data['lon']}")
#             else:
#                 print("Waiting for GNSS data...")

#             time.sleep(4)

# # Start the logging thread
# thread = threading.Thread(target=log_gnss_data, daemon=True)
# thread.start()

# Assuming 'gnss' is your actual GNSS object
# gnss.listen(_on_gnss_event)  # Uncomment when using a real GNSS object

# Function to choose a random coordinate from the file


In [ ]:
gnss.listen()

In [ ]:
def choose_random_coordinate():
    with open(log_file, "r") as file:
        lines = file.readlines()
        random_coordinate = random.choice(lines).strip()
        lat, lon = map(float, random_coordinate.split(","))
        print(f"Chosen Coordinate: Latitude: {lat}, Longitude: {lon}")
        return lat, lon


choose_random_coordinate()

In [ ]:
for sensor in world.get_blueprint_library().filter('*collision*'):
    print(sensor)
    x = sensor

In [ ]:
print(x)

In [ ]:
world_location = world.get_map().get_waypoint(vehicle.get_location()).transform.location
print(world_location)

In [1]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

NameError: name 'world' is not defined

In [ ]:
import carla
import time
import random

# ====== 1. Globals ======
latest_gnss = {'lat': 0.0, 'lon': 0.0}

# ====== 2. GNSS Sensor Callback ======
def gnss_callback(data):
    latest_gnss['lat'] = data.latitude
    latest_gnss['lon'] = data.longitude

# ====== 3. Attach GNSS Sensor to Vehicle ======
def attach_gnss_sensor(world, vehicle):
    blueprint_library = world.get_blueprint_library()
    gnss_bp = blueprint_library.find('sensor.other.gnss')
    gnss_bp.set_attribute('sensor_tick', '0.1')  # 10Hz
    spawn_point = carla.Transform(carla.Location(x=0.0, z=2.0))
    gnss_sensor = world.spawn_actor(gnss_bp, spawn_point, attach_to=vehicle)
    gnss_sensor.listen(gnss_callback)
    return gnss_sensor

# ====== 4. Choose Random Coordinate Offset ======
def choose_random_coordinate():
    # Small offset in degrees (~10–50 meters max)
    delta_lat = random.uniform(-0.0004, 0.0004)
    delta_lon = random.uniform(-0.0004, 0.0004)
    return delta_lat, delta_lon

def gnss_to_carla_location(lat, lon, vehicle, world):
    geo_ref = carla.GeoReference(world.get_map().transform_to_georeference())
    x, y = geo_ref.transform(lat, lon)
    z = vehicle.get_transform().location.z
    return carla.Location(x=x, y=y, z=z)

# ====== 6. Global Route Planner ======
from agents.navigation.global_route_planner import GlobalRoutePlanner

def create_global_planner(world, sampling_resolution=2.0):
    carla_map = world.get_map()
    planner = GlobalRoutePlanner(carla_map, sampling_resolution)
    planner.setup()
    return planner

# ====== 7. Plan Route from GNSS ======
def plan_route_from_ground_truth(world, vehicle):
    start_loc = vehicle.get_location()
    goal_loc = carla.Location(x=start_loc.x + 50.0, y=start_loc.y + 20.0, z=start_loc.z)

    dao = GlobalRoutePlannerDAO(world.get_map(), sampling_resolution=2.0)
    planner = GlobalRoutePlanner(dao)
    planner.setup()

    waypoints = planner.trace_route(start_loc, goal_loc)
    return waypoints, start_loc, goal_loc


# ====== 8. Draw Route (Debug) ======
def draw_route(world, waypoints, life_time=60.0):
    for wp in waypoints:
        world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(255,0,0), life_time=life_time)

# ====== 9. Main Flow ======
def run_global_planner(world, vehicle):
    gnss_sensor = attach_gnss_sensor(world, vehicle)
    print("Waiting for GNSS data...")
    time.sleep(1.5)  # Let sensor start

    waypoints, start_loc, goal_loc = plan_route_from_gnss(world, vehicle)
    draw_route(world, waypoints)
    print(f"Planned route from:\n{start_loc}\nto:\n{goal_loc}\nTotal waypoints: {len(waypoints)}")

    return waypoints, start_loc, goal_loc


In [105]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

vehicle = world.get_actors().filter("*vehicle*")[0]  # Use your own spawned vehicle

waypoints, start, goal = run_global_planner(world, vehicle)


Waiting for GNSS data...


AttributeError: module 'carla' has no attribute 'GeoReference'

In [12]:
# route planning bit like shown in Tutorial 3
import sys
sys.path.append('/home/kaustubh/CARLA_15/PythonAPI/carla/')

from agents.navigation.global_route_planner import GlobalRoutePlanner

point_a = start_point.location #we start at where the car is
print(point_a)
sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

# now let' pick the longest possible route
distance = 0
for loc in spawn_points: # we start trying all spawn points 
                            #but we just exclude first at zero index
    cur_route = grp.trace_route(point_a, loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route
#draw the route in sim window - Note it does not get into the camera of the car
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=10.0,
        persistent_lines=True)

Location(x=-64.644844, y=24.471010, z=0.600000)


In [13]:
vehicle_location = vehicle.get_transform().location
vehicle_gnss = world.get_map().transform_to_geolocation(vehicle_location)
print(f"Vehicle GNSS: lat={vehicle_gnss.latitude}, lon={vehicle_gnss.longitude}")


Vehicle GNSS: lat=-0.0002198307656868792, lon=-0.0005807147886869425


In [20]:
start_location = world.get_map().get_spawn_points()[0].location
geo = world.get_map().transform_to_geolocation(start_location)

print(start_location)
print(geo)

Location(x=-64.644844, y=24.471010, z=0.600000)
GeoLocation(latitude=-0.000220, longitude=-0.000581, altitude=0.600000)


In [3]:
import sys
import math
import numpy as np
import carla
import time

sys.path.append('/home/kaustubh/CARLA_15/PythonAPI/carla/dist/carla-0.9.15-py3.7-linux-x86_64.egg')

# === Connect to server
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
carla_map = world.get_map()

# === Find vehicle
vehicle = world.get_actors().filter('vehicle.*')[0]

# === Get GNSS sensor
# gnss = [s for s in world.get_actors().filter('sensor.other.gnss') if s.parent == vehicle]
gnss = world.get_actors().filter("*gnss*")
if len(gnss) == 0:
    raise Exception("GNSS not found.")
gnss = gnss[0]

# === Listen to GNSS
gnss_data = None
def gnss_callback(self, event):
    gnss_data = {
        "latitude": event.latitude,
        "longitude": event.longitude
    }

gnss.listen(gnss_callback)

print("[INFO] Waiting for GNSS fix...")
while gnss_data is None:
    time.sleep(0.1)

# === Get GeoReference from map
geo_ref_str = carla_map.to_opendrive().split("geoReference>")[1].split("</geoReference")[0]
geo_tags = dict([tag.split('=') for tag in geo_ref_str.strip().replace('+', '').split()])

# === Convert GNSS → Location (manual projection using pyproj)
import pyproj
proj = pyproj.Proj(f"+proj=utm +zone={geo_tags['zone']} +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

x, y = proj(gnss_data.longitude, gnss_data.latitude)
z = gnss_data.altitude

origin_x = float(geo_tags['false_easting'])
origin_y = float(geo_tags['false_northing'])

carla_x = x - origin_x
carla_y = y - origin_y
carla_z = z  # Assuming flat terrain, alt might need tuning

start_loc = carla.Location(x=carla_x, y=carla_y, z=carla_z)
print("[INFO] CARLA Start Location:", start_loc)

# === Goal GNSS (same projection)
goal_lat, goal_lon = -0.0009250, -0.0003950
goal_x, goal_y = proj(goal_lon, goal_lat)

carla_goal_x = goal_x - origin_x
carla_goal_y = goal_y - origin_y
carla_goal_z = 0.0

goal_loc = carla.Location(x=carla_goal_x, y=carla_goal_y, z=carla_goal_z)
print("[INFO] CARLA Goal Location:", goal_loc)

# === Waypoints
start_wp = carla_map.get_waypoint(start_loc, project_to_road=True)
goal_wp = carla_map.get_waypoint(goal_loc, project_to_road=True)

grp = carla.GlobalRoutePlanner(carla_map, sampling_resolution=2.0)
route = grp.trace_route(start_wp.transform.location, goal_wp.transform.location)

for wp, _ in route:
    world.debug.draw_point(wp.transform.location + carla.Location(z=0.5), size=0.2, color=carla.Color(255, 0, 0), life_time=20.0)

print(f"[INFO] Route with {len(route)} waypoints plotted.")
gnss.stop()


ModuleNotFoundError: No module named 'carla'

In [5]:
import carla
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
m = world.get_map()

print(dir(m))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'cook_in_memory_map', 'generate_waypoints', 'get_all_landmarks', 'get_all_landmarks_from_id', 'get_all_landmarks_of_type', 'get_crosswalks', 'get_landmark_group', 'get_spawn_points', 'get_topology', 'get_waypoint', 'get_waypoint_xodr', 'name', 'save_to_disk', 'to_opendrive', 'transform_to_geolocation']


In [1]:
import carla
print(dir(carla.Map))


ModuleNotFoundError: No module named 'carla'

In [14]:
location = world.get_map()

In [18]:
gnss_data = None

def gnss_callback(data):
    global gnss_data
    gnss_data = data

bp = world.get_blueprint_library().filter('*gnss*')
# gnss = world.spawn_actor(bp, carla.Transform(), attach_to=vehicle)
gnss.listen(gnss_callback)

# Wait until data comes in
while gnss_data is None:
    time.sleep(0.1)

lat = gnss_data.latitude
lon = gnss_data.longitude
alt = gnss_data.altitude

carla_loc = vehicle.get_transform().location
print(f"GNSS = ({lat}, {lon}, {alt})")
print(f"Location = ({carla_loc.x}, {carla_loc.y}, {carla_loc.z})")


: 